In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_color_codes()
from google.colab import drive
drive.mount("/content/drive/")

orig_path = '/content/drive/My Drive/bird_classification/' + '/data/bird_dataset/'

## For reproductibility
torch.manual_seed(2)
np.random.seed(2)
random.seed(2)

Mounted at /content/drive/


In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
import torch.nn.functional as F
from torch.autograd import Variable
from tqdm import tqdm
import torchvision.transforms as transforms

# Utils functions

In [ ]:
def data_load(size=(300,300), interpolation=1, batch_size=16):
  data_transforms_train = transforms.Compose([
    transforms.Resize(size, interpolation=interpolation),  ## https://pillow.readthedocs.io/en/latest/handbook/concepts.html#filters
    transforms.RandomRotation(45),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ])
  data_transforms_val = transforms.Compose([
    transforms.Resize(size, interpolation=interpolation),  ## https://pillow.readthedocs.io/en/latest/handbook/concepts.html#filters
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
  train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(orig_path + '/train_images',
                         transform=data_transforms_train),
    batch_size=batch_size, shuffle=True)
  val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(orig_path + '/val_images',
                         transform=data_transforms_val),
    batch_size=batch_size, shuffle=False)
  return train_loader, val_loader

def visualize_history(loss_train, loss_val, acc_train, acc_val):
  # Visual comparison
  titles = ["Loss Evolution", "Accuracy Evolution"]
  y = [[loss_train, loss_val], [acc_train, acc_val]]
  label_loss = [['Train Loss', 'Val Loss'], ['Train Accuracy', 'Validation Accuracy']]
  ncols = len(titles)
  fig, axes = plt.subplots(1, ncols, figsize=(15, 5))
  steps = np.arange(1,len(loss_train))
  for idx, ax in enumerate(axes.flatten()):
    title = titles[idx]
    ax.plot(steps, y[idx][0], label=label_loss[idx][0], color='b')
    ax.plot(steps, y[idx][1], label=label_loss[idx][1], color='r')
    ax.set_title(title)
    ax.legend()
  fig.tight_layout()

In [ ]:
def train(epoch, model, train_loader):
    model.train()
    correct = 0
    loss_values = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))
        loss_values += loss.data.item()
    avg_acc = 100*correct / len(train_loader.dataset)
    avg_loss = loss_values / len(train_loader.dataset)
    print(f'Average Accuracy : {avg_acc}')
    return (avg_acc, avg_loss)
        

def validation(model, val_loader):
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        data, target = data.cuda(), target.cuda()
        output = model(data)
        validation_loss += criterion(output, target).data.item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    avg_acc = 100. * correct / len(val_loader.dataset)
    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        validation_loss, correct, len(val_loader.dataset),
        avg_acc))
    return (avg_acc, validation_loss)

# Interpolation

## BOX
Each pixel of source image contributes to one pixel of the destination image with identical weights.

In [ ]:
train_loader, val_loader = data_load()

In [ ]:
from torchvision.models import resnet152

In [ ]:
model_resnet = resnet152(pretrained=True)
i_max = len(list(model_resnet.parameters())) - 33
i = 0
for param in model_resnet.parameters():
  if i > i_max:
    break
  i += 1
  param.requires_grad = False
num_features = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_features, 20)
model_resnet.cuda()

In [ ]:
param_optimizer = list(model_resnet.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
lr = 0.001
optimizer = torch.optim.Adam(optimizer_grouped_parameters, lr=lr)

criterion = torch.nn.CrossEntropyLoss()

In [ ]:
acc_train = []
loss_train = []
acc_val = []
loss_val = []
best_acc_val = 45
for epoch in range(1, 12):
    acc_t, loss_t = train(epoch, model_resnet)
    acc_v, loss_v = validation(model_resnet)
    acc_train.append(acc_t.item())
    loss_train.append(loss_t)
    acc_val.append(acc_v.item())
    loss_val.append(loss_v)

In [ ]:
history_box = {'t_l': loss_train, 't_a' : acc_train, 'v_l': loss_val, 'v_a': acc_val}

## Nearest

In [ ]:
train_loader, val_loader = data_load(interpolation=0)
model_resnet = resnet152(pretrained=True)
i_max = len(list(model_resnet.parameters())) - 33
i = 0
for param in model_resnet.parameters():
  if i > i_max:
    break
  i += 1
  param.requires_grad = False
num_features = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_features, 20)
model_resnet.cuda()

In [ ]:
param_optimizer = list(model_resnet.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
lr = 0.001
optimizer = torch.optim.Adam(optimizer_grouped_parameters, lr=lr)

criterion = torch.nn.CrossEntropyLoss()

In [ ]:
acc_train = []
loss_train = []
acc_val = []
loss_val = []
best_acc_val = 45
for epoch in range(1,12):
    acc_t, loss_t = train(epoch, model_resnet)
    acc_v, loss_v = validation(model_resnet)
    acc_train.append(acc_t.item())
    loss_train.append(loss_t)
    acc_val.append(acc_v.item())
    loss_val.append(loss_v)

In [ ]:
history_near = {'t_l': loss_train, 't_a' : acc_train, 'v_l': loss_val, 'v_a': acc_val}

### Bilinear

In [ ]:
train_loader, val_loader = data_load(interpolation=2)
model_resnet = resnet152(pretrained=True)
i_max = len(list(model_resnet.parameters())) - 33
i = 0
for param in model_resnet.parameters():
  if i > i_max:
    break
  i += 1
  param.requires_grad = False
num_features = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_features, 20)
model_resnet.cuda()

In [ ]:
param_optimizer = list(model_resnet.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
lr = 0.001
optimizer = torch.optim.Adam(optimizer_grouped_parameters, lr=lr)

criterion = torch.nn.CrossEntropyLoss()

In [ ]:
acc_train = []
loss_train = []
acc_val = []
loss_val = []
best_acc_val = 45
for epoch in range(1,12):
    acc_t, loss_t = train(epoch, model_resnet)
    acc_v, loss_v = validation(model_resnet)
    acc_train.append(acc_t.item())
    loss_train.append(loss_t)
    acc_val.append(acc_v.item())
    loss_val.append(loss_v)

In [ ]:
history_bilinear = {'t_l': loss_train, 't_a' : acc_train, 'v_l': loss_val, 'v_a': acc_val}

Winner is cubic interpolation since it has achieve 86.4 % of accuracy.
Now let's see if we can improve the accuracy by changing de size of our image

# Size

## (64,64)

In [ ]:
train_loader, val_loader = data_load(size=(64,64), interpolation=4)
model_resnet = resnet152(pretrained=True)
i_max = len(list(model_resnet.parameters())) - 33
i = 0
for param in model_resnet.parameters():
  if i > i_max:
    break
  i += 1
  param.requires_grad = False
num_features = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_features, 20)
model_resnet.cuda()

In [ ]:
param_optimizer = list(model_resnet.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
lr = 0.001
optimizer = torch.optim.Adam(optimizer_grouped_parameters, lr=lr)

criterion = torch.nn.CrossEntropyLoss()

In [ ]:
acc_train = []
loss_train = []
acc_val = []
loss_val = []
best_acc_val = 0
for epoch in range(1,12):
    acc_t, loss_t = train(epoch, model_resnet)
    acc_v, loss_v = validation(model_resnet)
    acc_train.append(acc_t.item())
    loss_train.append(loss_t)
    acc_val.append(acc_v.item())
    loss_val.append(loss_v)
    if acc_v.item()>best_acc_val:
      best_acc_val = acc_v.item()

In [ ]:
history_64_64 = {'t_l': loss_train, 't_a' : acc_train, 'v_l': loss_val, 'v_a': acc_val}

### (128,128)

In [ ]:
train_loader, val_loader = data_load(size=(128,128), interpolation=4)
model_resnet = resnet152(pretrained=True)
i_max = len(list(model_resnet.parameters())) - 33
i = 0
for param in model_resnet.parameters():
  if i > i_max:
    break
  i += 1
  param.requires_grad = False
num_features = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_features, 20)
model_resnet.cuda()

In [ ]:
param_optimizer = list(model_resnet.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
lr = 0.001
optimizer = torch.optim.Adam(optimizer_grouped_parameters, lr=lr)

criterion = torch.nn.CrossEntropyLoss()

In [ ]:
acc_train = []
loss_train = []
acc_val = []
loss_val = []
best_acc_val = 0
for epoch in range(1,12):
    acc_t, loss_t = train(epoch, model_resnet,train_loader)
    acc_v, loss_v = validation(model_resnet, val_loader)
    acc_train.append(acc_t.item())
    loss_train.append(loss_t)
    acc_val.append(acc_v.item())
    loss_val.append(loss_v)
    if acc_v.item()>best_acc_val:
      best_acc_val = acc_v.item()

In [ ]:
history_128_128 = {'t_l': loss_train, 't_a' : acc_train, 'v_l': loss_val, 'v_a': acc_val}

### (224,224)

In [ ]:
train_loader, val_loader = data_load(size=(224,224), interpolation=4)
model_resnet = resnet152(pretrained=True)
i_max = len(list(model_resnet.parameters())) - 33
i = 0
for param in model_resnet.parameters():
  if i > i_max:
    break
  i += 1
  param.requires_grad = False
num_features = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_features, 20)
model_resnet.cuda()

In [ ]:
param_optimizer = list(model_resnet.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
lr = 0.001
optimizer = torch.optim.Adam(optimizer_grouped_parameters, lr=lr)

criterion = torch.nn.CrossEntropyLoss()

acc_train = []
loss_train = []
acc_val = []
loss_val = []
best_acc_val = 0
for epoch in range(1,12):
    acc_t, loss_t = train(epoch, model_resnet,train_loader)
    acc_v, loss_v = validation(model_resnet, val_loader)
    acc_train.append(acc_t.item())
    loss_train.append(loss_t)
    acc_val.append(acc_v.item())
    loss_val.append(loss_v)
    if acc_v.item()>best_acc_val:
      best_acc_val = acc_v.item()

In [ ]:
history_224_224 = {'t_l': loss_train, 't_a' : acc_train, 'v_l': loss_val, 'v_a': acc_val}

### (334,334)

In [ ]:
train_loader, val_loader = data_load(size=(334,334), interpolation=4)
model_resnet = resnet152(pretrained=True)
i_max = len(list(model_resnet.parameters())) - 33
i = 0
for param in model_resnet.parameters():
  if i > i_max:
    break
  i += 1
  param.requires_grad = False
num_features = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_features, 20)
model_resnet.cuda()

In [ ]:
param_optimizer = list(model_resnet.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
lr = 0.001
optimizer = torch.optim.Adam(optimizer_grouped_parameters, lr=lr)

criterion = torch.nn.CrossEntropyLoss()

acc_train = []
loss_train = []
acc_val = []
loss_val = []
best_acc_val = 0
for epoch in range(1,12):
    acc_t, loss_t = train(epoch, model_resnet,train_loader)
    acc_v, loss_v = validation(model_resnet, val_loader)
    acc_train.append(acc_t.item())
    loss_train.append(loss_t)
    acc_val.append(acc_v.item())
    loss_val.append(loss_v)
    if acc_v.item()>best_acc_val:
      best_acc_val = acc_v.item()

Train Epoch: 1 [0/1082 (0%)]	Loss: 3.027610
Train Epoch: 1 [320/1082 (29%)]	Loss: 1.964068
Train Epoch: 1 [640/1082 (59%)]	Loss: 1.549480
Train Epoch: 1 [960/1082 (88%)]	Loss: 1.679340
Average Accuracy : 44.17744827270508

Validation set: Average loss: 0.1165, Accuracy: 60/103 (58%)
Train Epoch: 2 [0/1082 (0%)]	Loss: 1.367237
Train Epoch: 2 [320/1082 (29%)]	Loss: 0.872879
Train Epoch: 2 [640/1082 (59%)]	Loss: 0.690016
Train Epoch: 2 [960/1082 (88%)]	Loss: 1.357473
Average Accuracy : 70.05545043945312

Validation set: Average loss: 0.0951, Accuracy: 59/103 (57%)
Train Epoch: 3 [0/1082 (0%)]	Loss: 0.938491
Train Epoch: 3 [320/1082 (29%)]	Loss: 0.922523
Train Epoch: 3 [640/1082 (59%)]	Loss: 0.962184
Train Epoch: 3 [960/1082 (88%)]	Loss: 0.989885
Average Accuracy : 72.18114471435547

Validation set: Average loss: 0.0791, Accuracy: 60/103 (58%)
Train Epoch: 4 [0/1082 (0%)]	Loss: 0.355374
Train Epoch: 4 [320/1082 (29%)]	Loss: 0.944697
Train Epoch: 4 [640/1082 (59%)]	Loss: 0.647367
Train Epoc

Winner : (334,334)